In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import re
import glob
from sklearn.model_selection import train_test_split

In [ ]:
# import zipfile
# with zipfile.ZipFile('../input/dogs-vs-cats/test1.zip', 'r') as zip_ref:
#     zip_ref.extractall('./')

In [ ]:
trainData = glob.glob('./train/*.jpg')

In [ ]:
def decodeImageLabels(filePath):
    bits = tf.io.read_file(filePath)
    bits = tf.image.decode_jpeg(bits, channels=3) / 255
    image = tf.image.resize(bits, size=[32,32])
    temp = tf.strings.split(filePath, sep='/')[-1]
    temp = tf.strings.split(temp, sep='.')[0]
#   1 for dog, 0 for cat
    label = 1
    if temp =='cat':
        label = 0
    label = tf.reshape(label, [])
    return image, label

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
ignore_order = tf.data.Options()
ignore_order.experimental_deterministic = False

In [ ]:
trainTestData, testTrainData = train_test_split(trainData, test_size=0.1)
trainValData, valTrainData = train_test_split(trainTestData, test_size=0.1)

In [ ]:
trainImages = tf.data.Dataset.from_tensor_slices(trainValData)
valImages = tf.data.Dataset.from_tensor_slices(valTrainData)

In [ ]:
trainImages = trainImages.with_options(ignore_order)
valImages = valImages.with_options(ignore_order)

In [ ]:
trainImages = trainImages.map(decodeImageLabels, num_parallel_calls=AUTOTUNE)
trainImages = trainImages.repeat()
trainImages = trainImages.batch(512)
valImages = valImages.map(decodeImageLabels, num_parallel_calls=AUTOTUNE)
valImages = valImages.batch(512)

In [ ]:
with tf.device('/device:GPU:0'):
  model = keras.models.Sequential([
    keras.layers.Conv2D(filters=16, kernel_size=7, activation='relu', padding='same', input_shape=[32,32,3]),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    keras.layers.MaxPool2D(4),
#     keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
#     keras.layers.MaxPool2D(2),
#     keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
#     keras.layers.MaxPool2D(2),
#     keras.layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='same'),
#     keras.layers.MaxPool2D(2),
    keras.layers.Flatten(),
#     keras.layers.Dense(512, activation='relu'),
#     keras.layers.Dense(256, activation='relu'),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(128, activation='relu'),
#     keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
#     keras.layers.Dropout(0.5),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
#     keras.layers.Dropout(0.5),
    # For multiple class checking use softmax else if for single class use only sigmoid.
    keras.layers.Dense(1, activation='sigmoid')
  ])
  optimizer = keras.optimizers.Adamax()
  model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy', tf.keras.metrics.AUC(name='auc'), tf.keras.metrics.Precision(name='precision')])
  history = model.fit(trainImages, verbose=1, steps_per_epoch=20, epochs=50, validation_data=valImages)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
testImg = tf.data.Dataset.from_tensor_slices(testTrainData)
testImg = testImg.with_options(ignore_order)
testImg = testImg.map(decodeImageLabels, num_parallel_calls=AUTOTUNE)
testImg = testImg.batch(512)
testImg = testImg.prefetch(AUTOTUNE)

In [ ]:
model.evaluate(testImg)